In [39]:
import glob
import os
import sys
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
from multiprocessing import Process
import gc

import flwr as fl
from flwr.server.strategy import FedAvg
import tensorflow as tf
import sklearn
import time

import numpy as np
import pandas as pd
from pandas import DataFrame

import collections

#!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from flwr.common.logger import log
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [40]:
# argumentos
n = len(sys.argv)
print("Total arguments passed:", n)
iteracoes = 0
cycle_index = 0
finalIterations = 0
checkpoint_iteration = 0
if(n > 0):
    for value in sys.argv:
        print("arg:", value)
        if("iterations=" in value):
            try:
                iteracoes = int(value.replace("iterations=",""))
            except:
                print("no")
        
        if("cycle=" in value):
            try:
                cycle_index = int(value.replace("cycle=",""))
            except:
                print("no")
        if("checkpoint_iteration=" in value):
            try:
                iteracoes = int(value.replace("iterations=",""))
            except:
                print("no")
print("iteracoes:",iteracoes)      
print("cycle:",cycle_index)   

Total arguments passed: 3
arg: /home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/lib/python3.9/site-packages/ipykernel_launcher.py
arg: -f
arg: /home/guilherme/.local/share/jupyter/runtime/kernel-980930ae-3b38-4aa0-8d2a-bcb9e3320608.json
iteracoes: 0
cycle: 0


In [41]:
# input folder
#inputFolders = "../02-transformed-data-new-testes/dados2019/"
inputFolderPath = "../data_2019_processed/"
baseFolderWeek = inputFolderPath+"w"

WEEKS_TEST= [3]
#WEEKS_TEST= [2]
#WEEKS_TRAIN = [0,1,2]
WEEKS_TRAIN = [0]


# General configuration
NUMBER_OF_ITERATIONS_FINAL = 200
NUMBER_OF_ITERATIONS_FINAL = 10
    
NUM_EPOCHS = 1
BATCH_SIZE = 32
VERBOSE = 0

# usado para experimentos
if(iteracoes > 0):
    NUMBER_OF_ITERATIONS_FINAL = iteracoes
    
NUMBER_OF_ITERATIONS = NUMBER_OF_ITERATIONS_FINAL

# output folder
outputFolder = "result_unbalanced_epoch_"+str(NUM_EPOCHS)+"_rounds_"+str(NUMBER_OF_ITERATIONS_FINAL)+"_cycle_"+str(cycle_index)

#outputFolder = "test_checkpoint"
checkPointFolder = outputFolder+"/checkpoints"

# last cycle
last_cycle_index = cycle_index - 1
lastCycleOutputFolder = "result_unbalanced_epoch_"+str(NUM_EPOCHS)+"_rounds_"+str(200)+"_cycle_"+str(last_cycle_index)
lastCycleOutputFolder = "result_unbalanced_epoch_"+str(NUM_EPOCHS)+"_rounds_"+str(NUMBER_OF_ITERATIONS_FINAL)+"_cycle_"+str(last_cycle_index)
iferredCycleDataFolder = lastCycleOutputFolder+"/inferred_datasets"

# train file name modifier
fileSufixTrain = "_transformed" # _transformed_smote for smote

fl.common.logger.configure(identifier="myFlowerExperiment", filename="log_"+outputFolder+".txt")

# usado para checkpoints
if(checkpoint_iteration > 0):
    NUMBER_OF_ITERATIONS_FINAL = checkpoint_iteration
    
NUMBER_OF_ITERATIONS = NUMBER_OF_ITERATIONS_FINAL

In [42]:
print("Checking whether the folder exists or not")
isExist = os.path.exists(outputFolder)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(outputFolder)
    print("The new directory is created!")
else:
    print("The directory exists!")

Checking whether the folder exists or not
The directory exists!


In [43]:
print("Checking whether the checkpoint folder exists or not")
isExist = os.path.exists(checkPointFolder)
if not isExist:
    # Create a new directory because it does not exist
    os.makedirs(checkPointFolder)
    print("The new checkpoint directory is created!")
else:
    print("The checkpoint directory exists!")

Checking whether the checkpoint folder exists or not
The checkpoint directory exists!


In [44]:
print("check whether the cycle is 0 > or not, if so, the folder of inference must exist")
if(cycle_index > 0):
    isExist = os.path.exists(iferredCycleDataFolder)
    print(iferredCycleDataFolder)
    if not isExist:
        print("The folder of inference not exists!")
        sys.exit("The folder of inference not exists!")
    else:
        print("The checkpoint directory exists!")

check whether the cycle is 0 > or not, if so, the folder of inference must exist


In [45]:
# selected features
inputFeatures = ["activity","location","day_of_week","light","phone_lock","proximity","sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [46]:
# client datasets used on the training process (75% of data)
trainFolders =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                #'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                #'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4']
                #'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                #'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                #'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                #'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                #'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                #'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']
            
# client datasets used on the training process (25% of data)
testFolders =  [#'0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                #'0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                #'2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                #'2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                #'7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                #'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                #'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                #'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                #'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                #'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                #'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                #'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                #'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                #'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                #'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                #'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                #'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                #'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                #'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
                'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']

In [47]:
def generateMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    # confusion matrix
    matrix = confusion_matrix(y_test, yhat_classes)
    #print(matrix)
    
    array = []
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    results['matrix'] = ("[[ " +str(matrix[0][0]) + " " +str(matrix[0][1]) +"][ " +str(matrix[1][0]) + " " + str(matrix[1][1]) +"]]") # array.append(np.array(matrix,dtype=object))
    results['TP'] = matrix[0][0]
    results['FP'] = matrix[0][1]
    results['FN'] = matrix[1][0]
    results['TN'] = matrix[1][1]
    
    array.append(accuracy)
    array.append(precision)
    array.append(recall)
    array.append(f1)
    array.append(kappa)
    array.append(auc)
    array.append("[[ " +str(matrix[0][0]) + " " +str(matrix[0][1]) +"][ " +str(matrix[1][0]) + " " + str(matrix[1][1]) +"]]") # array.append(np.array(matrix,dtype=object))
    array.append(matrix[0][0]) # TP
    array.append(matrix[0][1]) # FP
    array.append(matrix[1][0]) # FN
    array.append(matrix[1][1]) # TN
    
    return results, array

# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # generate metrics
    results, array= generateMetrics(y_test,yhat_probs)

    # accuracy: (tp + tn) / (p + n)
    accuracy = results['accuracy']
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = results['precision']
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = results['recall'] 
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = results['f1_score']
    print('F1 score: %f' % f1)
    # kappa
    kappa = results['cohen_kappa_score']
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = results['roc_auc_score']
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("Confusion Matrix")
    matrix = results['matrix']
    print(matrix)
    
    return results, array

def generateGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score]

def showGlobalMetrics(metrics):
    res = generateGlobalMetrics(metrics)
    
    accuracy = res[0]
    precision = res[1]
    recall = res[2]
    f1_score = res[3]
    cohen_kappa_score = res[4]
    roc_auc_score = res[5]
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
    return res

In [48]:
# take the list of directories and concat them
def loadDataFromFolders(foldersToLoad,inputFolders,fileType = "_transformed"):
    print(len(foldersToLoad), "datasets")
    for i in range(0,len(foldersToLoad)):
        currentFolder = foldersToLoad[i]
        print(i , "-", currentFolder,inputFolders+"/student_"+currentFolder+""+fileType+".csv")
        #print(trainingDataSet[i])
        if(i == 0):
            temp_data = pd.read_csv(inputFolders+"/student_"+currentFolder+""+fileType+".csv")
        else:
            dataset = pd.read_csv(inputFolders+"/student_"+currentFolder+""+fileType+".csv")
            temp_data = pd.concat([temp_data, dataset])
    # return the dataset        
    return temp_data

# take the list of directories and concat them
def loadDataFromLastCycleFoldersOnList():
    clientList = []
    foldersToLoad = trainFolders
    print(len(foldersToLoad), "datasets")
    for i in range(0,len(foldersToLoad)):
        currentFolder = foldersToLoad[i]
        print(i , "-", currentFolder,iferredCycleDataFolder+"/student_"+currentFolder+".csv")
        #print(trainingDataSet[i])
        temp_data = pd.read_csv(iferredCycleDataFolder+"/student_"+currentFolder+".csv")
        temp_data['class'] = temp_data['awake']
        temp_data['class'] = temp_data['class'].astype('int32')
        
        mapper = {0: 'asleep', 1: 'awake'}

        temp_data['class'] = temp_data['class'].map(mapper)
        #temp_data['class'] = temp_data['class'].apply(lambda tpl: [mapper.get(x) for x in tpl])

        #temp_data['fullcoursenames'] = [[mapper.get(x) for x in tpl] for tpl in temp_data['itemsets']]

        #del temp_data['awake']
        #del temp_data['asleep']
        
        print("Adding to the list: ", temp_data.shape)
        clientList.append(temp_data)
    # return the dataset        
    return clientList

# take the list of directories and concat them
def loadDataFromFoldersOnList(foldersToLoad,inputFolders,fileType = "_transformed"):
    clientList = []
    print(len(foldersToLoad), "datasets")
    for i in range(0,len(foldersToLoad)):
        currentFolder = foldersToLoad[i]
        print(i , "-", currentFolder,inputFolders+"student_"+currentFolder+fileType+".csv")
        #print(trainingDataSet[i])
        temp_data = pd.read_csv(inputFolders+"student_"+currentFolder+fileType+".csv")
        print("Adding to the list: ", temp_data.shape)
        clientList.append(temp_data)
    # return the dataset        
    return clientList

In [59]:
print("Preparing test data")
 
# test data comprising 25% of the data. It must be fixed to all models being evaluated
#X_test  = pd.read_csv(inputFolders+"test/allData-classification-numeric-normalized.csv")
X_test = loadDataFromFolders(testFolders,inputFolderPath,"_transformed")

print()
# undestand the dataset by looking on their infos
print(X_test.info())

X_test

Preparing test data
8 datasets
0 - rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw ../data_2019_processed//student_rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw_transformed.csv
1 - RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI ../data_2019_processed//student_RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI_transformed.csv
2 - VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is ../data_2019_processed//student_VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is_transformed.csv
3 - Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw ../data_2019_processed//student_Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw_transformed.csv
4 - XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA ../data_2019_processed//student_XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA_transformed.csv
5 - YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw ../data_2019_processed//student_YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw_transformed.csv
6 - ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM ../data_2019_processed//student_ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM_transform

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.75,1.0,0.000000e+00,0.000000,0.515992,1.0,0.0,0.000000,1.0,0.678249,2018-05-14 16:16:08+00:00,awake
1,0.25,1.0,3.211282e-07,0.000000,0.542171,0.0,1.0,0.000007,1.0,0.678944,2018-05-14 16:17:39+00:00,awake
2,0.25,1.0,6.422564e-07,0.000000,0.515992,0.0,1.0,0.000000,1.0,0.679639,2018-05-14 16:18:39+00:00,awake
3,0.00,1.0,6.422564e-07,0.000000,0.515992,0.0,1.0,0.000000,1.0,0.680334,2018-05-14 16:19:09+00:00,awake
4,0.25,1.0,6.422564e-07,0.000000,0.531341,0.0,1.0,0.000000,1.0,0.681028,2018-05-14 16:20:09+00:00,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
13208,1.00,0.0,3.981050e-03,0.923909,0.577338,1.0,0.0,0.000266,1.0,0.035441,2018-06-05 00:51:21+00:00,awake
13209,1.00,0.0,3.981050e-03,0.923809,0.612109,1.0,0.0,0.000236,1.0,0.036136,2018-06-05 00:52:21+00:00,awake
13210,0.00,0.0,3.981371e-03,0.923611,0.573415,1.0,0.0,0.000118,1.0,0.037526,2018-06-05 00:54:11+00:00,awake
13211,0.75,0.0,3.981693e-03,0.923611,0.538685,1.0,0.0,0.000089,1.0,0.037526,2018-06-05 00:54:41+00:00,awake


In [60]:
print("Preparing X_train data")
# load cliend data
if(cycle_index == 0):
    print("Gething data from training folder")
    inputTempFolder = baseFolderWeek + "0/"
    clientList = loadDataFromFoldersOnList(trainFolders,inputTempFolder,fileSufixTrain)
    #clientList = loadDataFromFoldersOnList(trainFolders,inputFolderPath,fileSufixTrain)
else:
    print("Gething data from human inference folder")
    clientList = loadDataFromLastCycleFoldersOnList()

NUMBER_OF_CLIENTS = len(clientList)
print("Total",(len(clientList)))

Preparing X_train data
Gething data from training folder
19 datasets
0 - 0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs ../data_2019_processed/w0/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_transformed.csv
Adding to the list:  (1331, 12)
1 - 0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA ../data_2019_processed/w0/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_transformed.csv
Adding to the list:  (1331, 12)
2 - 2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0 ../data_2019_processed/w0/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_transformed.csv
Adding to the list:  (1331, 12)
3 - 2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys ../data_2019_processed/w0/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_transformed.csv
Adding to the list:  (1331, 12)
4 - 7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA ../data_2019_processed/w0/student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_transformed.csv
Adding to the list:  (1331, 12)
5 - a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4 ../data_2019_processed/w0/s

In [61]:
fileSufixTrain

'_transformed'

In [62]:
#clientList

In [63]:
#for t in clientList:
#    print(collections.Counter(t['class']))

In [64]:
#train =  loadDataFromFoldersOnList(trainFolders,inputFolderPath,fileSufixTrain)

In [65]:
#train

In [66]:
# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the input dataset
for i in range(0,len(clientList)):
    clientList[i] = transform_output_nominal_class_into_one_hot_encoding(clientList[i])
    #print (clientList[i])
    

X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110603 entries, 0 to 13212
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            110603 non-null  float64
 1   location            110603 non-null  float64
 2   timestamp           110603 non-null  float64
 3   time_to_next_alarm  110603 non-null  float64
 4   sound               110603 non-null  float64
 5   proximity           110603 non-null  float64
 6   phone_lock          110603 non-null  float64
 7   light               110603 non-null  float64
 8   day_of_week         110603 non-null  float64
 9   minutes_day         110603 non-null  float64
 10  timestamp_text      110603 non-null  object 
 11  class               110603 non-null  object 
 12  awake               110603 non-null  bool   
 13  asleep              110603 non-null  bool   
dtypes: bool(2), float64(10), object(2)
memory usage: 11.2+ MB


In [67]:
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# transforms the data
X_test = transform_data_type(X_test)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110603 entries, 0 to 13212
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            110603 non-null  float32
 1   location            110603 non-null  float32
 2   timestamp           110603 non-null  float64
 3   time_to_next_alarm  110603 non-null  float32
 4   sound               110603 non-null  float32
 5   proximity           110603 non-null  float32
 6   phone_lock          110603 non-null  float32
 7   light               110603 non-null  float32
 8   day_of_week         110603 non-null  float32
 9   minutes_day         110603 non-null  float32
 10  timestamp_text      110603 non-null  object 
 11  class               110603 non-null  object 
 12  awake               110603 non-null  float32
 13  asleep              110603 non-null  float32
dtypes: float32(11), float64(1), object(2)
memory usage: 8.0+ MB


In [68]:
print("Prepering the test dataset")
# selects the data to train and test
X_test_data = X_test[inputFeatures]
y_test_label = X_test[outputClasses]

# transtorm data to tensor slices
#client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data.values, y_test_label.values))

#client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True)
#client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)

#print(client_test_dataset.element_spec)
#client_test_dataset

Prepering the test dataset


In [69]:
#print("preparing the training datasets")
#federated_training_data = []
# transform the data
#for i in range(0,len(clientList)):
#    # selects the data to train and test
#    data   = clientList[i][inputFeatures]
#    labels = clientList[i][outputClasses]
#    # transform the data to tensor slices
#    client_train_dataset = tf.data.Dataset.from_tensor_slices((data.values, labels.values))
    # apply the configs
#    client_train_dataset = client_train_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)
    # transform the data to
 #   federated_training_data.append(client_train_dataset)

In [74]:
print("creating model")

def create_keras_model():
    return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(9,)),
      #tf.keras.layers.Dense(9, activation=tf.keras.activations.relu), 
      tf.keras.layers.Dense(16, activation=tf.keras.activations.relu),
      tf.keras.layers.Dense(8, activation=tf.keras.activations.relu),
      tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)
      #tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
    ])

keras_model = create_keras_model()
#keras_model.summary()
keras_model.summary()

creating model
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                160       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 2)                 18        
                                                                 
Total params: 314
Trainable params: 314
Non-trainable params: 0
_________________________________________________________________


In [75]:
# Load model and data (MobileNetV2, CIFAR-10)
#model = keras_model
#model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [76]:
def evaluate_and_save_results(keras_model,X_test_data, y_test_label, current_round_index, 
                              clientId, prefix_string = "Results", lossValue = -1):
     # predict values
    yhat_probs = keras_model.predict(X_test_data,verbose=VERBOSE)
    
    # as we deal with a classification problem with one hot encoding, we must round the values to 0 and 1.
    yhat_probs_rounded = yhat_probs.round()
    
    # create a dataframe with the predicted data
    y_predicted_df = pd.DataFrame(data=yhat_probs_rounded,columns=['awake','asleep']) 
    #y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 
    
    roundData = []

    columns = ['client','round','loss','class','accuracy','precision','recall', 
               'f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
               'TP','FP','FN','TN']
    
    # Instantiate the list that will contain the results
    listOfMetrics = list()
    
    #print('awake')    
    #res,resA = printMetrics(y_test_label['awake'],y_predicted_df['awake'])
    res,resA = generateMetrics(y_test_label['awake'],y_predicted_df['awake'])
    listOfMetrics.append(res)
    
    classData = np.concatenate(([clientId,current_round_index,lossValue,'awake'], resA))
    roundData.append(classData)
    
    #print('')
    #print('asleep')
    #res,resA = printMetrics(y_test_label['asleep'],y_predicted_df['asleep'])
    res,resA = generateMetrics(y_test_label['asleep'],y_predicted_df['asleep'])
    listOfMetrics.append(res)
    # new data
    classData = np.concatenate(([clientId,current_round_index,lossValue,'asleep'], resA))
    roundData.append(classData)
    
    #print('Global')
    #resA = showGlobalMetrics(listOfMetrics) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    resA = generateGlobalMetrics(listOfMetrics) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    # new data
    classData = np.concatenate(([clientId,current_round_index,lossValue,'avg'], resA))
    roundData.append(classData)
    
    dataMetrics = pd.DataFrame(data=roundData,columns=columns) 
    # write file
    if(clientId >= 0):
        outputMetricFile = outputFolder+"/"+prefix_string+"_MLP_client_" + str(clientId) + "_round_" + str(current_round_index) + ".csv"
    else:
        outputMetricFile = outputFolder+"/global_model_MLP_metrics.csv"
        outputMetricFile = outputFolder+"/"+prefix_string+".csv"
        # print global model results
        if(os.path.isfile(outputMetricFile)):
            dataset = pd.read_csv(outputMetricFile)
            dataMetrics = pd.concat([dataset, dataMetrics], axis=0)
        # Perform garbage collection
        gc.collect()
        
    dataMetrics.to_csv(outputMetricFile, sep=',', encoding='utf-8', index=False)

In [77]:
print("Loading checkpoint model",checkPointFolder+"/round-*")
list_of_files = [fname for fname in glob.glob(checkPointFolder+"/round-*")]
last_round_checkpoint = -1
latest_round_file = None
model_check_point = None
filename_np = None
filename_h5 = None

if len(list_of_files) > 0:
    latest_round_file = max(list_of_files, key=os.path.getctime)
    print("Loading pre-trained model from: ", latest_round_file)
    if(len(latest_round_file) > 0):
        # load the name
        last_round = latest_round_file.replace(checkPointFolder+"/round-","")
        last_round = last_round.replace("-weights.npz","")
        last_round = last_round.replace("-weights.h5","")
        print("Last round: ",last_round)
    
        last_round_checkpoint = int(last_round)
        filename_h5 = checkPointFolder+"/round-"+last_round+"-weights.h5"
        filename_np = checkPointFolder+"/round-"+last_round+"-weights.npz"
else:
    print("No checkpoint found")

    #check_point_model = tf.keras.models.load_model(latest_round_file)
    

Loading checkpoint model result_unbalanced_epoch_1_rounds_10_cycle_0/checkpoints/round-*
No checkpoint found


In [78]:
last_round_checkpoint

-1

In [79]:
#if latest_round_file is not None:
#    keras_model.load_weights(latest_round_file)

In [80]:
NUMBER_OF_ITERATIONS

10

In [81]:
if(last_round_checkpoint > -1):
    NUMBER_OF_ITERATIONS = NUMBER_OF_ITERATIONS_FINAL - (last_round_checkpoint)

    print("Number of iteractions after the checkpoint: ",NUMBER_OF_ITERATIONS)

In [82]:
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        
        #print("TEsteeee", aggregated_parameters)
        if aggregated_parameters is not None:
            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_ndarrays: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)

            # Save aggregated_ndarrays
            print(f"Saving round {server_round} aggregated_ndarrays...")
            fileName = f"{checkPointFolder}/round-{server_round}-weights.npz"
            print(fileName)
            #print(aggregated_parameters)
            print()
            np.savez(fileName, *aggregated_ndarrays)
            #np.savez(fileName+"2", *aggregated_parameters)
            #keras_model = create_keras_model()
            #keras_model.set_weights(aggregated_parameters)
            #keras_model.save_weights(fileName)

        return aggregated_parameters, aggregated_metrics

In [83]:
print("Declarating client function")

# Define a Flower client
class FlowerISABELASleepClient(fl.client.NumPyClient):

    def __init__(self, clientId, model, X_train_data, y_train_label,round_index=0):
        self.round_index = round_index
        self.clientId = clientId
        self.model = model
        self.X_train_data = X_train_data
        self.y_train_label = y_train_label


    def get_parameters(self, config):
        """Return current weights."""
        return self.model.get_weights()

    def fit(self, parameters, config):
               
        """Fit model and return new weights as well as number of training examples."""
        self.model.set_weights(parameters)
        
        
        # Evaluate local model parameters on the local test data
        loss, accuracy = self.model.evaluate(self.X_train_data, self.y_train_label,verbose=VERBOSE)       
        # print model results (verify the quality from the proxy data)
        evaluate_and_save_results(self.model,self.X_train_data, self.y_train_label, self.round_index, self.clientId,"local_data_before_fit",loss)
 
        # use the checkpoint if it is not -1
        #if(last_round_checkpoint == self.round_index and last_round_checkpoint != -1):
        #    print("loading checkpoint: ",filename_h5, " to client ",self.clientId)
        #    print("loading", latest_round_file)
        #    self.model = tf.keras.models.load_weights(filename_h5)
            
        self.model.fit(self.X_train_data, self.y_train_label, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,verbose=VERBOSE)

        # Evaluate local model parameters on the local test data
        loss, accuracy = self.model.evaluate(X_test_data, y_test_label,verbose=VERBOSE)

        # print model results
        evaluate_and_save_results(self.model,X_test_data, y_test_label, self.round_index, self.clientId,"proxy_data_after_fit",loss)
        return self.model.get_weights(), len(self.X_train_data), {}
    

Declarating client function


In [84]:
def get_evaluate_fn( model):
    
    def evaluate(
        server_round: int, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        
        current_round = server_round
        
        if(last_round_checkpoint > -1):
            current_round = server_round + last_round_checkpoint
            
        print("Evaluating global model round",current_round)
        
        model.set_weights(parameters)
        
        # Evaluate local model parameters on the local test data
        loss, accuracy = model.evaluate(X_test_data, y_test_label,verbose=VERBOSE)

        # only saves if the server_round + last_round_checkpoint != last_round_checkpoint to avaid duble metrics
        if(current_round > last_round_checkpoint):
            # print model results
            evaluate_and_save_results(model,X_test_data, y_test_label, current_round, -1,"global_model_metrics_after_agregation",loss)

            #checkpoint
            fileName = f"{checkPointFolder}/round-{current_round}-weights.h5"
            model.save_weights(fileName)
        else:
            print("Round already evaluated")
        
        return loss, { 'accuracy': accuracy }
    return evaluate

In [85]:
import ray
import math
# Create an instance of the model and get the parameters

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
#if DEVICE.type == "cuda":

client_resources = {"num_cpus": 1}

#keras_model = create_keras_model()
keras_model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])



def client_fn(cid) -> FlowerISABELASleepClient:
    print("starting client: "+str(cid),type(cid))
    #convert client ID to int
    clientId = int(cid)
    print("starting client: ", type(clientId))

    data   = clientList[clientId][inputFeatures]
    labels = clientList[clientId][outputClasses]
    
    print("Creating client model to client: "+str(cid))
    print("Data X: "+str(len(data)))
    print("Data Y: "+str(len(labels)))
    
    file_global_model = outputFolder+"/global_model_metrics_after_agregation.csv"
    index_round = 0 
    
    # get last
    if(os.path.isfile(file_global_model)):
        dataset = pd.read_csv(file_global_model)
        index_round = dataset["round"].max() + 1
        del dataset
    
    # update the index round in the previous checkpoint
    if(last_round_checkpoint > -1 and (index_round == last_round_checkpoint)):
        index_round = last_round_checkpoint
    
    print("Creating client model to client: "+str(cid),"round",index_round)
    # Load and compile a Keras model for CIFAR-10
    model = create_keras_model()
    model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
    
    proxyClient = FlowerISABELASleepClient(clientId,model,data,labels,index_round)
    
    return proxyClient

strategy = SaveModelStrategy(
    min_available_clients=NUMBER_OF_CLIENTS,
    evaluate_fn=get_evaluate_fn(keras_model)
) # (same arguments as FedAvg here)

# load checkpoint
if(filename_h5 is not None):
    
    #npzFile = np.load(filename_np)
    keras_model.load_weights(filename_h5)
    
    initial_parameters = keras_model.get_weights() 
    # Convert the weights (np.ndarray) to parameters (bytes)
    init_param = fl.common.ndarrays_to_parameters(initial_parameters)

    strategy = SaveModelStrategy(
        min_available_clients=NUMBER_OF_CLIENTS,
        evaluate_fn=get_evaluate_fn(keras_model),
        initial_parameters = init_param
    ) # (same arguments as FedAvg here)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUMBER_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUMBER_OF_ITERATIONS),  # Just three rounds
    client_resources=client_resources,
    strategy = strategy
)

INFO flwr 2024-03-05 15:02:25,532 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2024-03-05 15:02:28,542	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-05 15:02:29,950 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:172.30.126.159': 1.0, 'memory': 2047935284.0, 'object_store_memory': 1023967641.0, 'node:__internal_head__': 1.0, 'CPU': 16.0}
INFO flwr 2024-03-05 15:02:29,992 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-05 15:02:29,994 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO flwr 2024-03-05 15:02:30,011 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 16 actors
INFO flwr 2024-03-05 15:02:30,015 | server.py:89 | Initializing global parameters
INFO flwr 2024-03-05 15:02:30,018 | server.py:276 | Requesting initial parameters from one random clie

(DefaultActor pid=21516) starting client: 18 <class 'str'>
(DefaultActor pid=21516) starting client:  <class 'int'>
(DefaultActor pid=21516) Creating client model to client: 18
(DefaultActor pid=21516) Data X: 1331
(DefaultActor pid=21516) Data Y: 1331
(DefaultActor pid=21516) Creating client model to client: 18 round 0
Evaluating global model round 0


(DefaultActor pid=21516) /home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/lib/python3.9/site-packages/flwr/simulation/ray_transport/ray_actor.py:72: DeprecationWarning:  Ensure your client is of type `flwr.client.Client`. Please convert it using the `.to_client()` method before returning it in the `client_fn` you pass to `start_simulation`. We have applied this conversion on your behalf. Not returning a `Client` might trigger an error in future versions of Flower.
(DefaultActor pid=21516)   client = check_clientfn_returns_client(client_fn(cid))


2


INFO flwr 2024-03-05 15:02:46,247 | server.py:94 | initial parameters (loss, other metrics): 0.693403422832489, {'accuracy': 0.5219749808311462}
INFO flwr 2024-03-05 15:02:46,249 | server.py:104 | FL starting
DEBUG flwr 2024-03-05 15:02:46,250 | server.py:222 | fit_round 1: strategy sampled 19 clients (out of 19)
(DefaultActor pid=21516) /home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/lib/python3.9/site-packages/flwr/simulation/ray_transport/ray_actor.py:72: DeprecationWarning:  Ensure your client is of type `flwr.client.Client`. Please convert it using the `.to_client()` method before returning it in the `client_fn` you pass to `start_simulation`. We have applied this conversion on your behalf. Not returning a `Client` might trigger an error in future versions of Flower.
(DefaultActor pid=21516)   client = check_clientfn_returns_client(client_fn(cid))


...

ERROR flwr 2024-03-05 15:05:08,847 | ray_client_proxy.py:161 | Traceback (most recent call last):
  File "/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/lib/python3.9/site-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 151, in _submit_job
    res, updated_context = self.actor_pool.get_client_result(self.cid, timeout)
  File "/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/lib/python3.9/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 425, in get_client_result
    return self._fetch_future_result(cid)
  File "/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/lib/python3.9/site-packages/flwr/simulation/ray_transport/ray_actor.py", line 306, in _fetch_future_result
    res_cid, res, updated_context = ray.get(
  File "/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/lib/python3.9/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/gui

ERROR flwr 2024-03-05 15:05:08,850 | ray_client_proxy.py:162 | Task was killed due to the node running low on memory.
Memory on the node (IP: 172.30.126.159, ID: 899bcfe601613ee87600cac07e4408039ba67e531291030850a987fa) where the task (task ID: ffffffffffffffffad9c07d916b8fba81a3a59d401000000, name=DefaultActor.__init__, pid=21511, memory used=0.18GB) was running was 7.15GB / 7.44GB (0.960651), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: 7bb73a0a50885135ca02774680bc50484aa9fe62106a8041d4bc850e) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.30.126.159`. To see the logs of the worker, use `ray logs worker-7bb73a0a50885135ca02774680bc50484aa9fe62106a8041d4bc850e*out -ip 172.30.126.159. Top 10 memory users:
PID	MEM(GB)	COMMAND
10303	1.61	/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/bin/python -m ipykernel_launcher -f /h...
533	1.10	/home/guilherme/

ERROR flwr 2024-03-05 15:05:08,860 | ray_client_proxy.py:162 | Task was killed due to the node running low on memory.
Memory on the node (IP: 172.30.126.159, ID: 899bcfe601613ee87600cac07e4408039ba67e531291030850a987fa) where the task (actor ID: 54301ea457a4a9056ffe04a001000000, name=DefaultActor.__init__, pid=21499, memory used=0.22GB) was running was 7.07GB / 7.44GB (0.950464), which exceeds the memory usage threshold of 0.95. Ray killed this worker (ID: 109ad4743f450756e0f4847da2eba32e2672036bb77701c6b7b6c095) because it was the most recently scheduled task; to see more information about memory usage on this node, use `ray logs raylet.out -ip 172.30.126.159`. To see the logs of the worker, use `ray logs worker-109ad4743f450756e0f4847da2eba32e2672036bb77701c6b7b6c095*out -ip 172.30.126.159. Top 10 memory users:
PID	MEM(GB)	COMMAND
10303	1.61	/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/flower/bin/python -m ipykernel_launcher -f /h...
533	1.10	/home/guilherme/cpu-tensorflow-

(DefaultActor pid=21512) 2 [repeated 2x across cluster]
(DefaultActor pid=21506) starting client: 15 <class 'str'>
(DefaultActor pid=21506) starting client:  <class 'int'>
(DefaultActor pid=21506) Creating client model to client: 15
(DefaultActor pid=21506) Data X: 1331
(DefaultActor pid=21506) Data Y: 1331
(DefaultActor pid=21506) Creating client model to client: 15 round 11
(DefaultActor pid=21505) starting client: 9 <class 'str'>
(DefaultActor pid=21505) starting client:  <class 'int'>
(DefaultActor pid=21505) Creating client model to client: 9
(DefaultActor pid=21505) Data X: 1331
(DefaultActor pid=21505) Data Y: 1331
(DefaultActor pid=21505) Creating client model to client: 9 round 11


History (loss, centralized):
	round 0: 0.693403422832489
	round 1: 0.7716444134712219
	round 2: 0.7965250015258789
	round 3: 0.7837085127830505
	round 4: 0.785237729549408
	round 5: 0.7544503808021545
	round 6: 0.7448371648788452
	round 7: 0.7352955341339111
	round 8: 0.7206368446350098
	round 9: 0.7143182158470154
	round 10: 0.7127873301506042
History (metrics, centralized):
{'accuracy': [(0, 0.5219749808311462), (1, 0.3431371748447418), (2, 0.4252054691314697), (3, 0.46832364797592163), (4, 0.48932668566703796), (5, 0.5235030055046082), (6, 0.5410522222518921), (7, 0.559216320514679), (8, 0.5837092995643616), (9, 0.6002097725868225), (10, 0.6153992414474487)]}

In [86]:
clientList[0][inputFeatures]

,activity,location,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day
0,0.00,0.0,0.00,0.022970,0.0,1.0,0.000000,0.974501,0.681723
1,0.00,0.5,0.00,0.009981,0.0,1.0,0.604264,0.974402,0.682418
2,0.25,0.5,0.00,0.016817,0.0,1.0,0.597445,0.974303,0.683113
3,0.00,0.5,0.00,0.010254,0.0,1.0,0.597445,0.974303,0.683113
4,0.25,0.5,0.00,0.014629,0.0,1.0,0.536104,0.974204,0.683808
...,...,...,...,...,...,...,...,...,...
1326,1.00,0.0,0.25,0.011758,1.0,0.0,0.581536,0.988789,0.581654
1327,1.00,0.0,0.25,0.039923,0.0,1.0,0.557805,0.988689,0.582349
1328,0.75,0.0,0.25,0.006426,1.0,0.0,0.574526,0.988491,0.583739
1329,1.00,0.0,0.25,0.020782,0.0,1.0,0.558190,0.988293,0.585129


In [87]:
clientList[0][outputClasses]

,awake,asleep
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
1326,True,False
1327,True,False
1328,True,False
1329,True,False


(raylet) [2024-03-05 15:05:28,479 E 21098 21098] (raylet) node_manager.cc:3084: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 899bcfe601613ee87600cac07e4408039ba67e531291030850a987fa, IP: 172.30.126.159) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.30.126.159`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
